# Statistical bigram model

In this notebook, we will build our first language model. Large language models such as GPT generate text as a sequence of words. The language model presented here is very small and generates text as a sequence of individual characters (letters). Also, it is not built on a neural network but is based on probabilities defined on pairs of adjacent characters. Because of this, it is known as a **statistical bigram model**.

## Dataset

The task we will address in this notebook is to generate person names. The data for this task comes as a text file containing Swedish first names (*tilltalsnamn*). More specifically, the file lists the most frequent names in Sweden as of Decmber 2022 in decreasing order of frequency. The raw data was obtained from [Statistics Sweden](https://www.scb.se/) and postprocessed by lowercasing each name.

We start by opening the file and store its contents in a Python list:

In [ ]:
with open("names-train.txt", encoding="utf-8") as f:
    names = [line.rstrip() for line in f]

Here are the five most frequent names:

In [ ]:
names[:5]

### 🧩 Task 1: What’s in the data?

It is important to engage with the data you are working with. One way to do so is to ask questions. Is your own name included in the dataset? If so, is your name frequent or rare? Can you tell from a name whether the person with that name is male or female? Can you tell whether they are immigrants? What would be ethical and unethical uses of systems that *can* tell this?

## Character-to-index mapping

It will be convenient to represent characters by numbers. We therefore create a string-to-integer mapping from the characters in the names:

In [ ]:
char2idx = {"$": 0}

for name in names:
    for char in name:
        if char not in char2idx:
            char2idx[char] = len(char2idx)

Note that we reserve a special character `$` with index&nbsp;0. We will use this character to mark the start and the end of a sequence of characters.

### 🤯 What does this code do?

Throughout the course, we often present code without detailed explanations. If you need help understanding code, ask your tutor – or use an AI tool such as [ChatGPT](https://chatgpt.com/) and [Copilot](https://copilot.microsoft.com/). For example, the following explanation of the above code was generated by ChatGPT&nbsp;5 as a response to the prompt *Explain the following code in four sentences*.

> This code builds a dictionary mapping characters to unique integer indices. It starts with a special `$` character assigned to index `0`. Then, for every character in every string inside `names`, it checks if that character is already in the dictionary. If not, it assigns the character the next available index, which is simply the current dictionary length.

### 🧩 Task 2: A look inside the vocabulary

Write code that prints the vocabulary for the names dataset. Would you have expected this vocabulary? Would you expect the same vocabulary for a list of English names?

## Bigrams

As already mentioned, our model is based on pairs of consecutive characters (letters). Such pairs are called **bigrams**. For example, the character bigrams in the name `anna` are `an`, `nn`, and `na`. If we also include the start and end marker `$`, the bigrams are `$a`, `an`, `nn`, `na`, and `a$`.

The code in the following cell generates all character bigrams from a given iterable of names:

In [ ]:
def bigrams(names):
    for name in names:
        for x, y in zip("$" + name, name + "$"):
            yield x, y

For example, here are the bigrams extracted from the first two names in our dataset:

In [ ]:
[b for b in bigrams(names[:2])]

### 🤯 Generator functions

Note that `bigrams()` is a **generator function**: It does not `return` a list of all bigrams, it `yield`s all bigrams. This is more efficient in terms of memory usage, especially when dealing with the larger datasets we will encounter in this course. If you have not worked with generators and iterators before, now is a good time to read up on them. [More information about generators](https://wiki.python.org/moin/Generators)

## Estimating bigram probabilities

How does a language model generate text? Intuitively, we can imagine rolling a multi-sided dice whose sides correspond to the elements of the vocabulary. Whereas standard dice are fair (all sides land face up with the same uniform probability), the dice rolled by language models are weighted.

**The basic idea behind a bigram model is to let the probability of the next element in a generated sequence depend on the previous element (and only on that one).**

To estimate the probabilities of a bigram model, we start by counting how often each bigram occurs in the dataset. We can keep track of these counts by arranging them in a matrix&nbsp;$M$ that has as many rows and as many columns as there are characters in our vocabulary&nbsp;$V$. More formally, let $V = \{c_0, \dots, c_{n-1}\}$ where each $c_i$ is a character. Then the matrix entry $M_{ij}$ should be the count of the character bigram $c_ic_j$ in our list of names. To compute this matrix, we can use the code in the following cell:

In [ ]:
import torch

# Create a counts matrix with all zeros
counts = torch.zeros(len(char2idx), len(char2idx))

# Update the counts based on the bigrams
for x, y in bigrams(names):
    counts[char2idx[x], char2idx[y]] += 1

Note that we represent matrices as *tensors* from the [PyTorch library](https://pytorch.org/). You will learn more about this library later in the course.

Now that we have the bigram counts, we are ready to define our bigram model. This model is essentially a conditional probability distribution over all possible next characters, given the immediately preceding character. Formally, using the notation introduced above, the model consists of probabilities of the form $p(c_j\,|\,c_i)$, which quantify the probability of character&nbsp;$c_j$ given that the preceding character is&nbsp;$c_i$. To compute these probabilities, we divide each bigram count $M_{ij}$ by the sum along the $i$-th row of&nbsp;$M$. We can accomplish this as follows:

In [ ]:
model = counts / counts.sum(dim=-1, keepdim=True)

### 🧩 Task 3: Inspecting the model

According to this model, which letter is most likely to come after the letter `c`? Which letter is most likely to start or end a name? Can you give an example of a name that is impossible according to the model?

## Generating text

Now that we have estimated the probabilities of our bigram model, we can generate text. To do so, we repeatedly “roll a dice” by sampling from the next-character distribution, conditioning on the previous character. Equivalently, we can think of sampling from a collection of categorical distributions over the next character, one distribution per previous character.

In [ ]:
# Construct the inverse of the character-to-index mapping
idx2char = {i: c for c, i in char2idx.items()}

# Generate 5 samples
for _ in range(5):
    # We begin with the start-of-sequence marker
    generated = "$"

    while True:
        # Look up the integer index of the previous character
        previous_idx = char2idx[generated[-1]]

        # Get the relevant probability distribution
        probs = model[previous_idx]

        # Sample an index from the distribution
        next_idx = int(torch.multinomial(probs, num_samples=1).item())

        # Get the corresponding character
        next_char = idx2char[next_idx]

        # Break if the model generates the end-of-sequence marker
        if next_char == "$":
            break

        # Otherwise, add the next character to the output
        generated = generated + next_char

    # Print the generated output (without the start-of-sequence marker)
    print(generated[1:])

As we can see, the strings generated by our bigram model only vaguely resemble actual names. This should not really surprise us: after all, each next character is generated by only looking at the immediately preceding character, which is too short a context to model many important aspects of names.

### 🧩 Task 4: Probability of a name

What is the probability of our model generating your name? What is the probability of it generating the single-letter “name” `s`?

## Evaluating language models

Language models are commonly evaluated by computing their **perplexity**. Intuitively, this measures how “surprised” the model is about a given text. The higher the perplexity, the lower the probability which the model assigns to the text. To obtain the perplexity of a model on a given text, we first compute the average negative log-likelihood $-\log p(c_j|c_i)$ that the model assigns to each true next character $c_j$ given its predecessor $c_i$. Then, we exponentiate this average.

(In this course, the notation $\log x$ refers to the natural (base $e$) logarithm of&nbsp;$x$, which in other contexts is also written as $\ln x$ or $\log_e x$.)

The code in the next cell computes the perplexity of our bigram model on a held-out list of names. This list was sourced from [Wikipedia](https://en.wikipedia.org/w/index.php?title=Category:Swedish_given_names) and lightly edited.

In [ ]:
import math

# Read the test data
with open("names-test.txt", encoding="utf-8") as f:
    test_names = [line.rstrip() for line in f]

# Collect the negative log likelihoods
nlls = []
for prev_char, next_char in bigrams(test_names):
    nlls.append(-math.log(model[char2idx[prev_char], char2idx[next_char]]))

# Compute the perplexity
ppl = math.exp(sum(nlls) / len(nlls))

# Print the perplexity
print(f"{ppl:.1f}")

### 🧩 Task 5: Upper bound on perplexity

The perplexity of our bigram model can be interpreted as the average number of characters the model must choose from when trying to “guess” the reference text. The lowest possible perplexity value is&nbsp;1, which corresponds to the case where each next character is completely certain and no guessing is necessary. What is a reasonable *upper bound* on the perplexity of our bigram model?

That’s all folks!